In [1]:
import pandas as pd    #Data Cleaning
import numpy as np     #Array Calculation 
import matplotlib.pyplot as plt # Visualization
import seaborn as sns  #Advance analytical Visualization

import datetime as dt
import warnings # Uyarılar
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Table Content

In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Reading and copy of data
online_retail = pd.read_csv('/kaggle/input/yr2009-2010-retail-online-data/Year 2009-2010.csv',encoding= 'unicode_escape')
df = online_retail.copy()
df.head(11)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,12/1/2009 7:45,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,12/1/2009 7:45,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,12/1/2009 7:45,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,12/1/2009 7:46,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,12/1/2009 7:46,3.75,13085.0,United Kingdom


# Data Preparation and Preprocessing

In [4]:
def check_df(dataframe):
    print("################ Shape ####################")
    print(dataframe.shape)
    print("############### Columns ###################")
    print(dataframe.columns)
    print("############### Types #####################")
    print(dataframe.dtypes)
    print("############### Head ######################")
    print(dataframe.head())
    print("############### Tail ######################")
    print(dataframe.tail())
    print("############### Describe ###################")
    print(dataframe.describe().T)

check_df(df)

################ Shape ####################
(525461, 8)
############### Columns ###################
Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')
############### Types #####################
Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object
############### Head ######################
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

      InvoiceDate  Price  Customer ID         Country  

# Answering Some Questions About Data

In [5]:
#Are there any missing observations in the dataset?
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [6]:
#Remove the missing observations from the dataset
df.dropna(inplace=True)

In [7]:
df.isnull().sum() # for Checking now null values which is removed or not 

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [8]:
#How many unique items in the dataset?
df["Description"].nunique()

4459

In [9]:
#How many product in the dataset?
df["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    3245
REGENCY CAKESTAND 3 TIER              1872
STRAWBERRY CERAMIC TRINKET BOX        1536
ASSORTED COLOUR BIRD ORNAMENT         1376
HOME BUILDING BLOCK WORD              1229
                                      ... 
BLUE OWL DECORATION                      1
PINK OWL DECORATION                      1
IVORY WALL CLOCK                         1
RED WALL CLOCK                           1
BAKING MOULD EASTER EGG MILK CHOC        1
Name: Description, Length: 4459, dtype: int64

In [10]:
#Rank the 10 most ordered products from most to least
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending=False).head(10)

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,55861
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54274
BROCADE RING PURSE,47430
PACK OF 72 RETRO SPOT CAKE CASES,44507
ASSORTED COLOUR BIRD ORNAMENT,44120
60 TEATIME FAIRY CAKE CASES,35630
PACK OF 60 PINK PAISLEY CAKE CASES,30888
JUMBO BAG RED RETROSPOT,29498
BLACK AND WHITE PAISLEY FLOWER MUG,25679


In [11]:
#Remove the canceled transactions from the dataset
df = df[~df["Invoice"].str.contains("C", na=False)]

In [12]:
#The total amount of each invoice and we create that column just multiply 'Price' and 'Quantity'
df["TotalPrice"] = df["Quantity"] * df["Price"]

# Calculating of RFM Metrics

In [13]:
df.head(6)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom,30.0
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,12/1/2009 7:45,1.65,13085.0,United Kingdom,39.6


In [14]:
df.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
TotalPrice     float64
dtype: object

In [15]:
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"])

In [16]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

In [17]:
today_date = dt.datetime(2010, 12, 11)

In [18]:
today_date

datetime.datetime(2010, 12, 11, 0, 0)

# Recency

In [19]:
#Determining the analysis date for the recency 
df["InvoiceDate"].max()
today_date = dt.datetime(2010, 12, 11)
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: num.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [20]:
rfm

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93
...,...,...,...
18283.0,18,6,641.77
18284.0,67,1,461.68
18285.0,296,1,427.00


In [21]:
rfm.columns=["Recency","Frequency","Monetary"]
rfm

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93
...,...,...,...
18283.0,18,6,641.77
18284.0,67,1,461.68
18285.0,296,1,427.00


In [22]:
rfm = rfm[rfm["Monetary"] > 0]
rfm.describe()

,Recency,Frequency,Monetary
count,4312.000000,4312.000000,4312.000000
mean,91.172542,4.455705,2048.238236
std,96.861457,8.170213,8914.481280
min,1.000000,1.000000,2.950000
25%,18.000000,1.000000,307.987500
50%,53.000000,2.000000,706.020000
75%,136.000000,5.000000,1723.142500
max,374.000000,205.000000,349164.350000


# Converting RFM Scores to Single Variable

In [23]:
#Date from customer's last purchase.The nearest date gets 5 and the furthest date gets 1.
rfm["recency_score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Total number of purchases.The least frequency gets 1 and the maximum frequency gets 5.
rfm["frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

#Total spend by the customer.The least money gets 1, the most money gets 5.
rfm["monetary_score"]= pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.0,165,11,372.86,2,5,2
12347.0,3,2,1323.32,5,2,4
12348.0,74,1,222.16,2,1,1
12349.0,43,3,2671.14,3,3,5
12351.0,11,1,300.93,5,1,2


In [24]:
#RFM - The value of 2 different variables that were formed was recorded as a RFM_SCORE
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

# Segmenting Customers Using RFM Score

In [25]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.0,165,11,372.86,2,5,2,25,cant_loose
12347.0,3,2,1323.32,5,2,4,52,potential_loyalists
12348.0,74,1,222.16,2,1,1,21,hibernating
12349.0,43,3,2671.14,3,3,5,33,need_attention
12351.0,11,1,300.93,5,1,2,51,new_customers


# Interpretation of Descriptive Statistics of Segments

In [26]:
rfm[["segment", "Recency","Frequency","Monetary"]].groupby("segment").agg(["mean","count","max"]).round()

Recency            Frequency            Monetary        \
                       mean count  max      mean count  max     mean count   
segment                                                                      
about_to_sleep         54.0   343   71       1.0   343    2    441.0   343   
at_Risk               152.0   611  372       3.0   611    6   1189.0   611   
cant_loose            124.0    77  298       9.0    77   46   4099.0    77   
champions               7.0   663   15      13.0   663  205   6852.0   663   
hibernating           214.0  1015  374       1.0  1015    2    404.0  1015   
loyal_customers        36.0   742   71       7.0   742   42   2746.0   742   
need_attention         53.0   207   71       2.0   207    3   1060.0   207   
new_customers           9.0    50   15       1.0    50    1    386.0    50   
potential_loyalists    19.0   517   36       2.0   517    3    730.0   517   
promising              26.0    87   36       1.0    87    1    367.0    87   

                               
                          max  
segment                        
about_to_sleep         3502.0  
at_Risk               34095.0  
cant_loose            26287.0  
champions            349164.0  
hibernating           11881.0  
loyal_customers       50291.0  
need_attention        13545.0  
new_customers          2945.0  
potential_loyalists   12080.0  
promising              2390.0